# Packages

In [2]:
using FinEtools, FinEtoolsAcoustics, LinearAlgebra, StatsBase, HDF5

# Initial setup

In [3]:
rho = 1.21 * phun("kg/m^3") # mass density
c  = 343.0 * phun("m/s") # sound speed
bulk =  c ^ 2 * rho
frequency = 500.0 # frequency of the incident wave, Hz
omega = 2 * pi * frequency
dt = 1.0 / frequency / 10
tfinal = 115 * dt
nsteps = round(tfinal / dt) + 1
println("Step takes $(sciNotation(dt, 3)) s")
println("Total of $nsteps steps")
println("Final time: $(sciNotation(tfinal, 3)) s")

Step takes 2.0E-4 s
Total of 116.0 steps
Final time: 2.3E-2 s


# Mesh

Geometry

In [4]:
# list of meshes
Meshes = Array{Tuple{FENodeSet, AbstractFESet}, 1}()
# Q4 discretization of large rectangular domain divisions
push!(Meshes, Q4quadrilateral([0.0 0.0; 1.2 2.6], 12, 26)) # 1
push!(Meshes, Q4quadrilateral([0.0 2.6; 1.6 5.45], 15, 30)) # 2
push!(Meshes, Q4quadrilateral([1.35 0.0; 4.05 2.6], 30, 24)) # 3
push!(Meshes, Q4quadrilateral([1.6 2.6; 4.05 5.8], 25, 26)) # 4
push!(Meshes, Q4quadrilateral([4.25 1.2; 5.25 4.5], 10, 35)) # 5
push!(Meshes, Q4quadrilateral([4.05 4.5; 5.25 5.3], 12, 8)) # 6
push!(Meshes, Q4quadrilateral([4.25 5.3; 5.25 5.85], 10, 4)) # 7
push!(Meshes, Q4quadrilateral([5.25 1.2; 7.0 3.92], 22, 25)) # 8
# merge meshes
fens, outputfes = mergenmeshes(Meshes, 0.05)
# concatenate connectivities
fes = cat(outputfes[5], cat(outputfes[6], cat(outputfes[7], outputfes[8])))
fes = cat(outputfes[1], cat(outputfes[2], cat(outputfes[3], cat(outputfes[4], fes))))
println("$(size(fens.xyz, 1)) nodes")
println("$(size(fes.conn, 1)) elements")
println("$(round(Int, size(fens.xyz, 1) * nsteps)) dataset points (using Q4)")

3371 nodes
3168 elements
391036 dataset points (using Q4)


Subdomain selection

In [5]:
# Identify boundary finite element set
bfes = meshboundary(fes)
# In case there are any unconnected nodes, remove them, and renumber the elements.
connected = findunconnnodes(fens, fes)
fens, new_numbering = compactnodes(fens, connected)
fess = renumberconn!(fes, new_numbering)
# The geometry and the solution (pressure) fields
geom = NodalField(fens.xyz)
P = NodalField(zeros(FCplxFlt, size(fens.xyz, 1), 1));

Visualize domain

In [6]:
# Export three VTK files: one for the interior of the fluid, and one for the boundary
vtkexportmesh("interior.vtk", fes.conn, fens.xyz, FinEtools.MeshExportModule.VTK.Q4);

# Discrete model

Setup

In [7]:
# Number the degrees of freedom in the pressure field.
numberdofs!(P)
# Create the finite element machine for the fluid.
material = MatAcoustFluid(bulk, rho)
femm  =  FEMMAcoust(IntegDomain(fes, GaussRule(3, 2)), material)
# Use the machine calculate the acoustic stiffness and mass matrices.
S  =  acousticstiffness(femm, geom, P)
C  =  acousticmass(femm, geom, P)
# "damping" matrix for the absorbing boundary conditions (ABCs)
D  =  acousticABC(
  FEMMAcoustSurf(IntegDomain(bfes, GaussRule(2, 2)), material), geom, P
)
loadNode = subset(fes, selectelem(fens, fes, withnodes = [2049, 2050, 2075, 2074]))
dipfemm = FEMMAcoustSurf(IntegDomain(loadNode, GaussRule(2, 2)), material)
# harmonic point load
function pointLoad(dpdn, xyz, J, label, t)::Float64
  dpdn[1] = -rho * 0.02 * sin(omega * t)
end;

Solution

In [8]:
# Solve the transient acoustics equations.
# The loop executes inside this local scope
t = 0.0 # Initial time
# initialize variable for dataset.
# dataset is (N x 4) matrix with rows: [nodeX nodeY time nodalPressure]ₙ
# N is the amount of samples: number of nodes * number of time steps
data = Float32[0.0 0.0 0.0 0.0]
pressureMat = zeros(Float32, (nnodes(P), 1))
P1 = let
  P0 = deepcopy(P)
  P0.values .= 0.0 # initially all pressure is zero
  vP0 = gathersysvec(P0)
  vQ0 = zeros(eltype(vP0), size(vP0))
  # The `P1` field will be the output of this computation:
  # the final value of the pressure field
  P1 = deepcopy(P0)
  fi = ForceIntensity( # Initial load
    FCplxFlt, 1, (dpdn, xyz, J, label) -> pointLoad(dpdn, xyz, J, label, t)
  )
  La0 = distribloads(dipfemm, geom, P1, fi, 2)
  A = (2.0 / dt) * S + D + (dt / 2.0) * C
  step = 0
  while t <= tfinal
    step += 1
    global t += dt
    # Update load
    fi = ForceIntensity( # Initial load
      FCplxFlt, 1, (dpdn, xyz, J, label) -> pointLoad(dpdn, xyz, J, label, t)
    )
    La1 = distribloads(dipfemm, geom, P1, fi, 2)
    # Solve for the rate of the pressure
    vQ1 = A \ ((2 / dt) * (S * vQ0) - D * vQ0 - C * (2 * vP0 + (dt / 2) * vQ0) + La0 + La1)
    
    # Update the value of the pressure
    vP1 = vP0 + (dt / 2) * (vQ0 + vQ1)
    
    # Swap variables for next step  
    vP0 = deepcopy(vP1)
    vQ0 = deepcopy(vQ1)
    P1 = scattersysvec!(P1, vec(vP1))
    P0 = deepcopy(P1)
    La0 = deepcopy(La1)
    # concatenate current pressure field to dataset.
    global data = vcat(
      data, hcat(fens.xyz, fill(t, size(fens.xyz, 1)), real.(P1.values) |> vec)
    )
    global pressureMat = hcat(pressureMat, real.(P1.values))
  end
  P1 # Return the final pressure
end
# discard matrix initializations
data = data[2:end, :]
pressureMat = pressureMat[:, 2:end];

Export pressure fields

In [9]:
for fileID in axes(pressureMat, 2)
  vtkexportmesh(
    "./vtkFiles/wavePropagation$fileID.vtk",
    fes.conn, geom.values, FinEtools.MeshExportModule.VTK.Q4;
    scalars = [( "pressure", pressureMat[:, fileID] |> vec)]
  )
end;

# Data

Summary statistics

In [10]:
for i in 1:4
  @show data[:, i] |> unique |> length
  statsum(data[:, i])
  println()
end

(data[:, i] |> unique) |> length = 309


Summary Stats:
Length:         391036
Missing Count:  0
Mean:           3.156618
Minimum:        0.000000
1st Quartile:   1.493333
Median:         3.060000
3rd Quartile:   4.750000
Maximum:        7.000000
Standard deviation: 1.9564E0
384424/391036 (98.3%) non-zero elements./n

(data[:, i] |> unique) |> length = 370
Summary Stats:
Length:         391036
Missing Count:  0
Mean:           2.829628
Minimum:        0.000000
1st Quartile:   1.625000
Median:         2.790000
3rd Quartile:   4.025000
Maximum:        5.850000
Standard deviation: 1.5305E0
385932/391036 (98.7%) non-zero elements./n



(data[:, i] |> unique) |> length = 116
Summary Stats:
Length:         391036
Missing Count:  0
Mean:           0.011700
Minimum:        0.000200
1st Quartile:   0.005950
Median:         0.011700
3rd Quartile:   0.017450
Maximum:        0.023200
Standard deviation: 6.697E-3
391036/391036 (100.0%) non-zero elements./n

(data[:, i] |> unique) |> length = 391036
Summary Stats:
Length:         391036
Missing Count:  0
Mean:           -0.000000
Minimum:        -0.000036
1st Quartile:   -0.000001
Median:         -0.000000
3rd Quartile:   0.000000
Maximum:        0.000028
Standard deviation: 3.2227E-6
391036/391036 (100.0%) non-zero elements./n



Normalize

In [11]:
# standardize
standardData = standardize(ZScoreTransform, data, dims = 1)
# unit normalization
unitData = standardize(UnitRangeTransform, data, dims = 1)

391036×4 Matrix{Float64}:
 0.0        0.0       0.0  0.562352
 0.0142857  0.0       0.0  0.562352
 0.0285714  0.0       0.0  0.562352
 0.0428571  0.0       0.0  0.562352
 0.0571429  0.0       0.0  0.562352
 0.0714286  0.0       0.0  0.562352
 0.0857143  0.0       0.0  0.562352
 0.1        0.0       0.0  0.562352
 0.114286   0.0       0.0  0.562352
 0.128571   0.0       0.0  0.562352
 ⋮                         
 0.909091   0.670085  1.0  0.559429
 0.920455   0.670085  1.0  0.559281
 0.931818   0.670085  1.0  0.559237
 0.943182   0.670085  1.0  0.559315
 0.954545   0.670085  1.0  0.559443
 0.965909   0.670085  1.0  0.559538
 0.977273   0.670085  1.0  0.559584
 0.988636   0.670085  1.0  0.559609
 1.0        0.670085  1.0  0.559655

Summary statistics of standardized data

In [16]:
mapslices(statsum, standardData; dims = [1]);

Summary Stats:
Length:         391036
Missing Count:  0
Mean:           0.000000
Minimum:        -1.613522
1st Quartile:   -0.850197
Median:         -0.049387
3rd Quartile:   0.814466
Maximum:        1.964565
Standard deviation: 1.0E0
391036/391036 (100.0%) non-zero elements./n
Summary Stats:
Length:         391036
Missing Count:  0
Mean:           0.000000
Minimum:        -1.848812
1st Quartile:   -0.787075
Median:         -0.025892
3rd Quartile:   0.781028
Maximum:        1.973439
Standard deviation: 1.0E0
391036/391036 (100.0%) non-zero elements./n
Summary Stats:
Length:         391036
Missing Count:  0
Mean:           0.000000
Minimum:        -1.717181
1st Quartile:   -0.858590
Median:         0.000000
3rd Quartile:   0.858590
Maximum:        1.717181
Standard deviation: 1.0E0
391036/391036 (100.0%) non-zero elements./n
Summary Stats:
Length:         391036
Missing Count:  0
Mean:           -0.000000
Minimum:        -11.039268
1st Quartile:   -0.059022
Median:         0.107731
3rd 

Save in HDF5

In [13]:
# create file
h5open("./dataset.h5", "w") do dataFile
  dataFile["data"] = data # [x y t p]ₙ data
  dataFile["unitData"] = unitData # unit normalized [x y t p]ₙ data
  # standardized [x y t p]ₙ data
  dataFile["standardData"] = standardData
  # [p₁ p₂ ... pₜ] evolution of pressure field
  dataFile["pressureSteps"] = pressureMat
end;

Summary statistics of file

In [14]:
h5open("./dataset.h5", "r") do f
  for field in HDF5.get_datasets(f)
    @show HDF5.name(field)[2 : end]
    d = HDF5.read(field)
    @show size(d)
    statsum(d)
    println()
  end
end

(HDF5.name(field))[2:end] = "data"


size(d) = (391036, 4)


Summary Stats:
Length:         1564144
Missing Count:  0
Mean:           1.499486
Minimum:        -0.000036
1st Quartile:   0.000006
Median:         0.022600
3rd Quartile:   2.880000
Maximum:        7.000000
Standard deviation: 1.946E0
1552428/1564144 (99.3%) non-zero elements./n

(HDF5.name(field))[2:end] = "pressureSteps"
size(d) = (3371, 116)
Summary Stats:
Length:         391036
Missing Count:  0
Mean:           -0.000000
Minimum:        -0.000036
1st Quartile:   -0.000001
Median:         -0.000000
3rd Quartile:   0.000000
Maximum:        0.000028
Standard deviation: 3.2227E-6
391036/391036 (100.0%) non-zero elements./n

(HDF5.name(field))[2:end] = "standardData"
size(d) = (391036, 4)


Summary Stats:
Length:         1564144
Missing Count:  0
Mean:           0.000000
Minimum:        -11.039268
1st Quartile:   -0.716293
Median:         0.091213
3rd Quartile:   0.661119
Maximum:        8.782824
Standard deviation: 10.0E-1
1564144/1564144 (100.0%) non-zero elements./n

(HDF5.name(field))[2:end] = "unitData"
size(d) = (391036, 4)
Summary Stats:
Length:         1564144
Missing Count:  0
Mean:           0.497890
Minimum:        0.000000
1st Quartile:   0.319934
Median:         0.550344
3rd Quartile:   0.635714
Maximum:        1.000000
Standard deviation: 2.4484E-1


1549056/1564144 (99.0%) non-zero elements./n

